EMBEDDINGS TO FETCH THE STORIES


In [1]:
import json
from pathlib import Path
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [2]:
import json

def load_json(file_path, type_name):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        for entry in data:
            entry["type"] = type_name
        return data

# Example: loading legends
legends = load_json("data/legends.json", "legend")
short_stories = load_json("data/short_stories.json", "short_story")

all_stories = legends + short_stories

print(f"Loaded {len(all_stories)} stories.")


Loaded 72 stories.


In [3]:
texts = [f"{s['title']}\n{s['narration']}\nLesson: {s['lesson']}" for s in all_stories]
metadatas = [{"title": s["title"], "type": s["type"], "lesson": s["lesson"]} for s in all_stories]


In [5]:
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Ensure all metadata values are strings, numbers, bool, or None
def sanitize_metadata(meta):
    sanitized = {}
    for k, v in meta.items():
        if isinstance(v, list):
            sanitized[k] = " | ".join(str(i) for i in v)  # convert list to string
        elif isinstance(v, dict):
            sanitized[k] = str(v)  # convert dict to string
        else:
            sanitized[k] = v
    return sanitized

sanitized_metadatas = [sanitize_metadata(m) for m in metadatas]

# Create or load the Chroma vectorstore
vectorstore = Chroma.from_texts(
    texts,
    embeddings,
    metadatas=sanitized_metadatas,
    persist_directory="story_db"
)

print("✅ Embeddings created & saved safely in 'story_db/'.")


c:\Users\General Happiness\Desktop\MY_AGENT\rag-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Embeddings created & saved safely in 'story_db/'.


In [7]:
query = "a story about bravery"
results = vectorstore.similarity_search(query, k=2)

for r in results:
    print(f"📖 {r.metadata['title']} ({r.metadata['type']})")
    print(r.page_content[:300], "...\n")


📖 The Story of Courageous Nakku (legend)
The Story of Courageous Nakku
Long ago, in a village surrounded by thick forests and wide grasslands, there lived a young woman named Nakku. She was known among her people for her quiet strength, though she was not one to boast about it.

One season, a terrible famine struck the land. Mothers cried  ...

📖 The Story of Courageous Nakku (legend)
The Story of Courageous Nakku
Long ago, in a village surrounded by thick forests and wide grasslands, there lived a young woman named Nakku. She was known among her people for her quiet strength, though she was not one to boast about it.

One season, a terrible famine struck the land. Mothers cried  ...

